# Subtyping and Variance Annotations

We will now explore the concept of _subtyping_ and the use of variance annotations in scala. 

## Subtypes

In Lettuce, we studied types such as __num__, __bool__, __num => bool__, __num => (num => num)__ and so on. 
However, these types are not related to each other:  __num__ represents numbers, __bool__ represents booleans, __num => (num => num)__ represents functions that take in numbers and return a function from number to number (remember currying). Each type represents a set of possible values that belong to the type, but  is _disjoint_ from the other types.

However, the situation is different when we define objects with the inheritance relationship between them. Each class we define is a type, however, when a class B inherits from class A, we are establishing a relationship between the types defined by these classes.  We call this relation the __subtyping relation__: B is a subtype of A. 
 
For the purposes of this lecture, we will _define_ subtyping in the following manner: 

__Liskov's Substitution Principle:__ 

A type $t_2$ is a subtype of type $t_1$ if in any function where an argument of type $t_1$ is input, an element of type $t_2$ can be supplied in its place.


Let us study this with examples. We define a class hierarchy as shown below.

In [1]:
abstract class Animal{
    val name: String
    val numLegs: Int
}
class Bird(val name: String) extends Animal {
    val numLegs: Int = 2
}

class Mammal(val name: String, val numLegs: Int) extends Animal{
}

class Rodent(override val name: String) extends Mammal(name, numLegs=4) { 
    override val numLegs= 4
}

class Cat(override val name: String) extends Mammal(name, numLegs = 4)


defined class Animal
defined class Bird
defined class Mammal
defined class Rodent
defined class Cat

Thus in this example, `Cat` is a subtype of `Mammal`. `Mammal` is a subtype of `Animal`. 
Inheritance of classes is one way of creating subtyping relationships. _We will study other subtype relationships later_.

What does the Liskov substitution principle mean in this context? 

In any function that takes as an input an argument of type `Animal`, we can substitute instead an argument of type `Bird`, `Mammal`, `Rodent` or even a `Cat`.


In [2]:
def sayHelloTo(a: Animal): String = "Hello, "+a.name

defined function sayHelloTo

In [3]:
sayHelloTo(new Cat("Macavity"))

res2: String = "Hello, Macavity"

In [4]:
sayHelloTo(new Rodent("Topo Gigio"))

res3: String = "Hello, Topo Gigio"

The function `sayHelloTo` defined above takes in an Animal `a` as an input. We can instead provide it a Cat, Rodent, a Mammal or any of its subtypes.
However, the other way round is not possible. I.e, we cannot necessarily substitute a supertype where a subtype is needed.

In [5]:
def meow(c: Cat): String = "Meow, "+c.name

defined function meow

In [26]:
val a: Animal = new Rodent("Paws") 

a: Animal = ammonite.$sess.cmd0$Helper$Rodent@645f1b88

In [26]:
meow(a) 
// This will fail because an Animal object cannot be used where a 
// Cat object is expected.


cmd26.sc:1: type mismatch;
 found   : ammonite.$sess.cmd25.wrapper.cmd0.Animal
 required: ammonite.$sess.cmd4.wrapper.cmd0.Cat
val res26 = meow(a) 
                 ^Compilation Failed

: 

## Subtyping in Scala

Scala is an object oriented language based on Java. Therefore, it implictly defines a class hierarchy that is baked in. For instance, any object that is defined by us including `Animal`, `Cat`, `Mammal` etc are subtypes of a type called `AnyRef`, which itself is a subtype of `Any`.

The class hierarchy of scala can be seen as below:

<img src="https://www.scala-lang.org/files/archive/spec/2.12/public/images/classhierarchy.png">



## Subtyping Relations between Function Types

Now that we have created subtyping relationships between objects such as `Animal`, `Bird`, `Mammal` , `Rodent` and `Cat`, let us turn our attention to function types.

Take two functions: `foo: String => Mammal` and `bar: String => Cat`. 
What is the subtyping relationship between the type of `foo` and the type of  `bar`?

Let us explore this concept a little better. We define a function mammalFactory that takes in as input a function foo of type `String => Mammal`.

In [7]:
def mammalFactory(foo: String => Mammal) = {
    foo("Lilibeth")
}

defined function mammalFactory

In [8]:
mammalFactory( (x: String) => new Mammal(x, 2))

res7: Mammal = ammonite.$sess.cmd0$Helper$Mammal@380c1bb3

Can we instead call mammalFactory with a function of type `String => Cat`? 


In [9]:
mammalFactory( (x: String) => new Cat(x))

res8: Mammal = ammonite.$sess.cmd0$Helper$Cat@65bfb601

It works. In fact, it is easy to convince onself that in any context where a function of type `String => Mammal` is asked for, we can always substitute a function of type `String => Cat`. But the other way round will not work. If we need a function from `String => Mammal`, can we instead pass a function from `String => Animal` instead?

In [9]:
mammalFactory ( (x: String) => new Rodent(x).asInstanceOf[Animal]  )

cmd9.sc:1: type mismatch;
 found   : cmd9.this.cmd0.Animal
 required: ammonite.$sess.cmd6.wrapper.cmd0.Mammal
val res9 = mammalFactory ( (x: String) => new Rodent(x).asInstanceOf[Animal]  )
                                                                    ^Compilation Failed

: 

The answer is that it does not work. 

### An Alternative View

Here is an easy way of thinking about it. Functions are things that take inputs and provide output. Suppose your code __requires__ a function `foo` that takes in _flour_ and bakes a _cake_, whereas you __provide__ a function `bar` that takes in _flour_ and bakes a _icedChocoloateCake_.

~~~
Require: Flour => Cake

Provided: Flour => IcedChocoloateCake
~~~

Let us assume that an `IcedChocolateCake` is a _subtype_ of `Cake`. 

Is it acceptable to provide a function of type `Flour => IcedChocoloateCake` where one is expecting a function of type `Flour => Cake`?

Yes of course. An IcedChocolateCake is after all a Cake, only yummier.
This kind of relationship is called a _covariant relationship_ (covariant: varies in the same direction as).

Thus, we can establish the following rule:

~~~
If B is a subtype of A then any function `T => B` is a subtype of function `T => A`
~~~



As an example: `Mammal` is a subtype of `Animal`. Therefore, `Int => Mammal` is a subtype of `Int => Animal`.

`String` is a subtype of `Any`. Therefore: `Int => String` is a subtype of `Int => Any`.

<img src="covariantRelationship.png" width=40%>



### Subtyping rule for function types (part 1)

Subtyping relation between functions is _covariant_ in the return type.

## Subtyping for Inputs of Functions

Let us consider the reverse situation now. You are asked to implement a pretty printer object that will take a given document type and print it out as a string.



In [10]:
abstract class AllDocuments
class WordDocument extends AllDocuments
class PDFDocument extends AllDocuments
class TextDocument extends AllDocuments

defined class AllDocuments
defined class WordDocument
defined class PDFDocument
defined class TextDocument

I am interested in a pretty printer function  of the following type

~~~
Expected: AllDocuments => String
~~~

You provide me with a function instead of type

~~~
Provided: PDFDocument => String
~~~

Will the provided function type be a substitute for the expected function type?



In [11]:
def printWithPrinterFun( pFun: AllDocuments => String) = {
    // Blah blah blah
}

defined function printWithPrinterFun

In [12]:
def myFancyPDFPrinter(p: PDFDocument ): String = {
    // .. print the pdf document to a string
    return "final printed document"
}

defined function myFancyPDFPrinter

In [12]:
printWithPrinterFun(myFancyPDFPrinter)

cmd12.sc:1: type mismatch;
 found   : ammonite.$sess.cmd11.wrapper.cmd9.PDFDocument => String
 required: ammonite.$sess.cmd10.wrapper.cmd9.AllDocuments => String
val res12 = printWithPrinterFun(myFancyPDFPrinter)
                                ^Compilation Failed

: 

We get an error when we try to pass `PDFDocument => String` type to a function that takes in `AllDocuments => String` as an input. What is happening here?

It turns out that a function  `AllDocuments => String` is more specialized or detailed than a function `PDFDocument => String` since the former can print all documents including PDF documents, whereas the latter can only print pdf documents.

### Subtyping rule for function types (part 2)

If `B` is a subtype of `A` then `A => T` is a subtype of `B => T`.

<img src="contravarianceIllustr.png" width=40%>

We say that subtyping between functions is __contravariant__ in the type of the input.


### Subtyping rule for functions (full rule)

Suppose `B` is a subtype of `A` and `D` is a subtype of `C` then 
`A => D` is a subtype of `B => C`. Subtyping is covariant in the return type and contravariant in the input type of the functions.

<img src="functionSubtypingIllustr.png" width="40%">

This rule is a simple combination of the two situations explained previously.

# Invariance, CoVariance and Contravariance for Generics

Now we consider the behavior of classes that take in type parameters or generics.

Consider the simple example of the `List[T]` class in scala.

Suppose we define a list `List[Cat]`. Can we view this as a `List[Animal]`? It makes sense that every `Cat` is a subtype of `Animal`. Therefore a `List[Cat]` is a subtype of `List[Animal]`.

In [13]:
def totalNumLegs(l: List[Animal]): Int = {
    l.foldLeft(0)((acc, elt) => acc + elt.numLegs)
}

defined function totalNumLegs

In [14]:
val l1: List[Cat] = List(new Cat("Jo"), new Cat("Bo"), new Cat("Mr.Jenkins"))

l1: List[Cat] = List(
  ammonite.$sess.cmd0$Helper$Cat@6585e03a,
  ammonite.$sess.cmd0$Helper$Cat@f7baa32,
  ammonite.$sess.cmd0$Helper$Cat@132169b6
)

In [15]:
print(totalNumLegs(l1))

12

It works! We could pass an object of type `List[Cat]` to a function where the function was expecting an object of type `List[Animal]`. 

Similarly, it seems that we can do the same trick for an Array of Cats vs. Array of Animals?

In [16]:
def totalNumLegsArray(a: Array[Animal]) = {
    a.foldLeft (0) {(b, elt) => b + elt.numLegs}
}

defined function totalNumLegsArray

In [17]:
val array1 = new Array[Cat](3)
array1(0)= new Cat("Jo")
array1(1) = new Cat("Bo")
array1(2) = new Cat("Mr.Jenkins")

array1: Array[Cat] = Array(
  ammonite.$sess.cmd0$Helper$Cat@5f9c4fa,
  ammonite.$sess.cmd0$Helper$Cat@15375cae,
  ammonite.$sess.cmd0$Helper$Cat@37113982
)

In [17]:
totalNumLegsArray(array1)

cmd17.sc:1: type mismatch;
 found   : Array[ammonite.$sess.cmd16.wrapper.cmd0.Cat]
 required: Array[ammonite.$sess.cmd15.wrapper.cmd0.Animal]
Note: ammonite.$sess.cmd16.wrapper.cmd0.Cat <: ammonite.$sess.cmd15.wrapper.cmd0.Animal, but class Array is invariant in type T.
You may wish to investigate a wildcard type such as `_ <: ammonite.$sess.cmd15.wrapper.cmd0.Animal`. (SLS 3.2.10)
val res17 = totalNumLegsArray(array1)
                              ^Compilation Failed

: 

It failed! Why is scala able to view a `List[Cat]` as a subtype of `List[Animal]`, whereas it cannot view an `Array[Cat]` as a subtype of `Array[Animal]`?

## Covariant, Contravariant and Invariant 

It turns out that an object `List[T]` in scala is _covariant_ in the typeof `T`. This means that : 

~~~
If B is a subtype of A then List[B] is a subtype of List[A]
~~~

<img src="listSubtypingIllustr.png" width=50%>

However, `Array[T]` (for very good reasons we will explain) is not _covariant_. It is in fact _invariant_. 

~~~
If B is a subtype of A, then Array[B] has no subtyping relation to Array[A].
~~~

<img src="arrayInvarianceIllustr.png" width=50%>

Finally,  we can have a contravariant relationship.

### Contravariant 

Let us consider a class `PrettyPrinter[T]` that prints a type out as a string. It turns out that if we implemented a `PrettyPrinter[Any]` that would be a huge deal since it can pretty print anything. However, `PrettyPrinter[Char]` is not as big a deal since it only pretty prints `Char`. Thus the type `PrettyPrinter[T]` is _contravariant_ in T.

<img src="prettyPrinterContraIllustr.png" width="50%">


## Variance Annotations

Suppose I would like to design my own class 
`SriramsList[T]` and make it covariant on the type parameter `T`, we do it in scala as follows.

In [18]:
abstract class SriramsList[+T] { // the + in front of the T says that it is covariant in T
    //.. members defined here
    def head: T
    def isEmpty: Boolean
    // ... other members here
}

defined class SriramsList

The way we specify covariance is to put a `+` in front of the type parameter __T__

However, it is not simply sufficient that we put a `+` in front of T. We have to make sure that the type parameter T appears only appears in  _covariant_ positions in the class. 

In [18]:
abstract class Covey[+T] {
    val x: T //T can appear as a field in a class
    def fun1(x: Int): T // T can appear as a return value of a function
    // FORBIDDEN
    def fun2(y: T): Int // T cannot appear as an argument to a function in the class. 
}

cmd18.sc:5: covariant type T occurs in contravariant position in type T of value y
    def fun2(y: T): Int // T cannot appear as an argument to a function in the class. 
             ^Compilation Failed

: 

As you can see from the error above, when we declare the class `Covey[+T]`, it is expected that 
~~~
T can only appear in covariant positions:
- immutable members of the class
- output types of functions
~~~
However, `T` cannot appear as an input type to a function. These are called `contravariant` positions. 

Can you guess why the covariant and contravariant positions so named? Go back to the discussion of covariance/contravariance of function types.

In [19]:
abstract class Covey[+T] {
    val x: T //T can appear as a field in a class
    def fun1(x: Int): T // T can appear as a return value of a function
    //def fun2(y: T): Int -- No: we cannot have T appear in a contravariant position
}

defined class Covey

Likewise for a contravariant class, we note that `T` cannot appear in a covariant position and can only appear in contravariant positions.

In [19]:
abstract class PrettyPrinter[-T] {  // - in front of T specifies that it is contravariant
   val x: T // FORBIDDEN -- covariant position
}


cmd19.sc:2: contravariant type T occurs in covariant position in type => T of value x
   val x: T // FORBIDDEN -- covariant position
       ^Compilation Failed

: 

In [19]:
abstract class PrettyPrinter[-T] {  // - in front of T specifies that it is contravariant
   def fun1(x: Int): T // Forbidden covariant position
}


cmd19.sc:2: contravariant type T occurs in covariant position in type (x: Int)T of method fun1
   def fun1(x: Int): T // Forbidden covariant position
       ^Compilation Failed

: 

In [20]:
abstract class PrettyPrinter[-T] {  // - in front of T specifies that it is contravariant
   def doPrinting(t: T): String // OK -- T is in a contravariant position
}

defined class PrettyPrinter

To specify that a class is invariant on T, we simply drop the + or - in front of T

~~~
abstract class Invar[T] { // No + or - in front of T says that it is invariant.
    // now there are no restrictions in what kind of positions that T may appear in.
    def x(i: T): T // Both covariant and contravariant positions -- OK
    def y(z: Int): T // Covariant position -- OK
    val x: T // Covariant --OK
    def contra(x: T): Int // Contravariant -- OK
}
~~~

In [21]:
abstract class Invar[T] { // No + or - in front of T says that it is invariant.
    // now there are no restrictions in what kind of positions that T may appear in.
    def x(i: T): T // Both covariant and contravariant positions -- OK
    def y(z: Int): T // Covariant position -- OK
    val x: T // Covariant --OK
    def contra(x: T): Int // Contravariant -- OK
}

defined class Invar

Let us now try to complete the definition of `SriramsList`.

In [21]:
abstract class SriramsList[+T] { // the + in front of the T says that it is covariant in T
    //.. members defined here
    def head: T
    def isEmpty: Boolean
    def addElement(t: T): SriramsList[T]
}

cmd21.sc:5: covariant type T occurs in contravariant position in type T of value t
    def addElement(t: T): SriramsList[T]
                   ^Compilation Failed

: 

We tried to add a function `addElement` and it broke the covariant annotation. Can you explain why `addElement` is problematic?

How then can we add an element to our list? We do it in a different way.

In [22]:
abstract class SriramsList[+T] { // the + in front of the T says that it is covariant in T
    //.. members defined here
    def head: T
    def isEmpty: Boolean
    def addElement[S >: T] (s : S): SriramsList[S]
}

defined class SriramsList

We use the following trick

~~~
def addElement[S >: T] (s : S): SriramsList[S]
~~~

It is a function that allows us to add an element of type `S`, where `T` is a subtype of `S` and the resultng list becomes of type `SriramsList[S]`.

In fact this behavior is seen in the scala list as well.

In [23]:
val l1: List[Cat] = List( new Cat("Mr. Meow"), new Cat("Jenkins"), new Cat("Mr. Mistofelles"))

l1: List[Cat] = List(
  ammonite.$sess.cmd0$Helper$Cat@2acb897e,
  ammonite.$sess.cmd0$Helper$Cat@1e69eb14,
  ammonite.$sess.cmd0$Helper$Cat@54c59c0
)

In [24]:
val l2 = (new Rodent("Crookshanks")):: l1  

l2: List[Mammal] = List(
  ammonite.$sess.cmd0$Helper$Rodent@51057e01,
  ammonite.$sess.cmd0$Helper$Cat@2acb897e,
  ammonite.$sess.cmd0$Helper$Cat@1e69eb14,
  ammonite.$sess.cmd0$Helper$Cat@54c59c0
)

We added a `Rodent` to our list and see how the entire list is now type promoted to a new list of type `Mammal`

## Mutable vars cannot be used in covariant/contravariant

Let us examine why Arrays in scala must be invariant. The reason is because they are mutable. And as we will see, one cannot use a mutable of type T if we would like to be co/contravariant on T. 


In [24]:
abstract class WithMutable[+T](initVal: T) {
  var mutableField: T = initVal
}

cmd24.sc:2: covariant type T occurs in contravariant position in type T of value mutableField_=
  var mutableField: T = initVal
      ^Compilation Failed

: 

As you can see scala does not allow it. Why is that? Suppose scala would allow it? Then we can break type soundness quite easily and do arbitrary typecasting as follows.

~~~
class WithMutable[+T] (initVal: T) {
  var mutableField: T = initVal
  def getMutableField: T = {mutableField}
}


val a1: WithMutable[Int] = WithMutable[Int](25) // mutableField is set to 25

val a2: WithMutable[Any] = a1 // This is OK since WithMutable is Co-Variant
a2.mutableField = "Mistofelles" // This is OK since String is a subtype of Any.

// But a2 and a1 are the same memory in scala. 

val x = a1.getMutableField // What is the type of x? Is it Int or String?
// Type system is broken since we wrote a string and try to get it out as an Int
~~~

## Rule for Mutable Vars

A class `Generic[T]` which contains a mutable field of type `T` cannot be declared co-variant or contravariant. It can only be invariant.